In [ ]:
import matplotlib.pyplot as plt
from librosa.display import specshow
import torch
from modules import mel2wav_vocoder
from models.models_HiFi import Generator as HiFiGAN
from modules import AttrDict
import torchaudio
import os
import json


### **plot_mel 함수**
- predict mel과 target mel을 시각화해서 비교해볼 수 있음

In [ ]:
def plot_mel(predict_mel, target_mel, sr=22050, hop_length=256):
    """
    predict mel vs target mel을 시각화해서 볼 수 있는 함수

    returns: 시각화 출력
    """
    predict_mel = predict_mel.detach().cpu().numpy()
    target_mel = target_mel.detach().cpu().numpy()

    fig, axes = plt.subplots(1,2,figsize=(10,5))

    # 예측 이미지
    predict_img = specshow(predict_mel, x_axis="time", y_axis='mel', sr=sr,
                           hop_length=hop_length, ax=axes[0], cmap='viridis')
    fig.colorbar(predict_img, ax=axes[0], format="%+2.0f DB")
    axes[0].set(title="Predicted Mel")


    # 타겟 이미지
    target_img = specshow(target_mel,x_axis="time", y_axis='mel', sr=sr,
                           hop_length=hop_length, ax=axes[1] cmap='viridis' )
    fig.colorbar(predict_img, ax=axes[0], format="%+2.0f DB")
    axes[1].set(title="Target Mel")

    #plot
    plt.tight_layout()
    plt.show()

### **generate_wav_vocoder 함수**
- vocoder(사전학습완료된) 사용해서 멜 스펙트로그램 -> 음성으로 변환하는 함수

In [ ]:
def generate_wav_vocoder(mel_spectrogram, vocoder_path, output_sr=16000):
    """
    보코더 사용해서 멜 스펙트로그램 -> 음성 변환하는 함수
    args:
        mel_spectrogram: 멜 스펙트로그램(80,172) 형태가 될 것
        vocoder_path: HiFiGAN 사용예정

    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # vocoder 설정
    vocoder_config_path = os.path.join(os.path.driname(vocoder_path), "config.json")
    with open(vocoder_config_path, "r") as f:
        config = json.load(f)
    h= AttrDict(config)
    vocoder = HiFiGAN(h).to(device)

    # pretrained model 가중치 로드
    state_dict = torch.load(vocoder_path, map_location = device)
    vocoder.load_state_dict(state_dict['generator'])
    vocoder.eval()

    # 음성으로 변환
    audio = mel2wav_vocoder(mel_spectrogram, vocoder, 1)

    if output_sr != 22050:
        audio = torchaudio.tunctional.resample(audio, original_freq = 22050, new_freq= output_sr)
    return audio
        